In [1]:
!pip install torch


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install chess

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import numpy as numpy # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
import chess

In [ ]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:  
                break
            games.append(game)
    return games

files = [file for file in os.listdir("../pgn") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 16)

games = []
for file in tqdm(files[:LIMIT_OF_FILES]):  
    games.extend(load_pgn(f"../pgn/{file}")) #Liste içinde liste olmaması için append yerine extend kullanıldı.

print(f"Toplam {len(games)} oyun yüklendi.")


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:06<00:00,  2.35it/s]

Toplam 3356 oyun yüklendi.


In [ ]:
from funcs import create_input_for_nn, encode_moves

In [6]:
x, y = create_input_for_nn(games)

print(f"Number of samples: {len(y)}")

Number of samples: 283583


In [7]:
x = x[0:280000]
Y = y[0:280000]

In [8]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [9]:
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [10]:
from dataset import ChessDataset
from model import ChessModel

In [11]:
dataset = ChessDataset(x, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader): .4f}, Time: {minutes}m{seconds}s")

100%|██████████| 4375/4375 [06:11<00:00, 11.78it/s]


Epoch 51/101, Loss:  5.3328, Time: 6m11s


100%|██████████| 4375/4375 [24:19<00:00,  3.00it/s]


Epoch 52/101, Loss:  4.1085, Time: 24m19s


100%|██████████| 4375/4375 [20:18<00:00,  3.59it/s]


...

Epoch 98/101, Loss:  0.3476, Time: 3m20s


100%|██████████| 4375/4375 [03:22<00:00, 21.65it/s]


Epoch 99/101, Loss:  0.3381, Time: 3m22s


100%|██████████| 4375/4375 [03:23<00:00, 21.51it/s]

Epoch 100/101, Loss:  0.3299, Time: 3m23s


In [14]:
torch.save(model.state_dict(), "./models/Torch_50epochs.pth")

In [15]:
import pickle

with open("./models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)